In [1]:
import pandas as pd

In [2]:
import glob

In [3]:
import numpy as np

In [4]:
import os

In [5]:
def scaling(input_data, offset, adjustment_factor):
    return (
        np.log(
            (input_data - offset)
            .clip(1e-6))
        .multiply(adjustment_factor)
    )

In [6]:
def revenue_ratios_new_plants(average_expected_revenue):
    
    revenue_lifetime_new = pd.Series(index = average_expected_revenue.filter(like="new").columns)

    lifetime_new = costs.loc[:, "lifetime", :].reindex([i.split(" ")[1] for i in revenue_lifetime_new.index]).value
    lifetime_new.index = revenue_lifetime_new.index

    for plant in revenue_lifetime_new.index:

        tech = initial_capacity_table.loc[plant, "carrier"].iloc[0]
        entry = initial_capacity_table.loc[plant, "entry"].iloc[0]

        initial_capacity_table.loc[plant]

        lifetime = lifetime_new.loc[plant]

        revenue_horizon = average_expected_revenue[plant].loc[entry: min(end_year, entry+lifetime)].sum()
        revenue_beyond = average_expected_revenue.loc[end_year,plant]*max(lifetime - (end_year - entry), 0)
        revenue_lifetime_new.loc[plant] = revenue_horizon + revenue_beyond

    invest_cost_new = costs.loc[:, "investment", :].reindex([i.split(" ")[1] for i in revenue_lifetime_new.index]).value.multiply(1e3) #converstion to EUR/MW
    invest_cost_new.index = revenue_lifetime_new.index




    fixed_om_new = (
        costs.loc[:, "FOM", :]
        .reindex([i.split(" ")[1] for i in revenue_lifetime_new.index])
        .value
        .multiply(
            costs.loc[:, "investment", :]
            .reindex([i.split(" ")[1] for i in revenue_lifetime_new.index])
            .value
            .multiply(1e3)
        )
    )

    fixed_om_new.index = revenue_lifetime_new.index

    total_fixed_new = invest_cost_new + fixed_om_new.multiply(lifetime_new)

    return revenue_lifetime_new.div(total_fixed_new)

In [7]:
def revenue_ratios_existing_plants(average_expected_revenue):
    

    revenue_lifetime_existing = pd.DataFrame(
        index = average_expected_revenue.index, 
        columns = [i for i in average_expected_revenue if "new" not in i]
    )

    relative_fixed_om = costs.loc[:, "FOM", :].reindex([i.split(" ")[1] for i in revenue_lifetime_existing]).value

    relative_fixed_om.index = revenue_lifetime_existing.columns

    invest_cost = (
        costs.loc[:, "investment", :]
        .reindex([i.split(" ")[1] for i in revenue_lifetime_existing])
        .value
        .multiply(1e3) #converstion to EUR/MW
    ) 

    invest_cost.index = revenue_lifetime_existing.columns
    total_fixed_om = relative_fixed_om.multiply(invest_cost).divide(100) # O&M given in %
    

    total_fixed = pd.DataFrame(
            index = average_expected_revenue.index, 
            columns = [i for i in average_expected_revenue if "new" not in i]
        )

    for plant in revenue_lifetime_existing.columns:
        
        tech = initial_capacity_table.loc[plant, "carrier"].unique()[0]

        entry = int(initial_capacity_table.loc[plant, "entry"].unique()[0])
        exit = int(initial_capacity_table.loc[plant, "exit"].unique()[0])

        for year in range(max(start_year, entry), min(exit, end_year+1)):
            revenue_lifetime_existing.loc[year, plant] = average_expected_revenue.loc[start_year:year, plant].fillna(0).sum()
            total_fixed.loc[year, plant] = (year + 1 - entry)*total_fixed_om.loc[plant]

    revenue_lifetime_existing = revenue_lifetime_existing.astype(float)
    total_fixed = total_fixed.astype(float)

    return revenue_lifetime_existing.divide(total_fixed)
        

In [8]:
adjustment_factor = 0.005

In [9]:
offset = 0.1

In [10]:
demand = pd.read_hdf("resources/demand.h5")

In [11]:
revenue_files = glob.glob("results/revenues/0/*")

In [12]:
initial_capacity_table = pd.read_csv("resources/capacity_tables/0.csv", index_col=[0,1])

In [13]:
revenues = []
climate_years = []
target_years = []

In [14]:
for file in revenue_files:
    revenues.append(pd.read_hdf(file))

    climate_years.append(int(file.split("cy")[1][:4]))
    target_years.append(int(file.split("ty")[1][:4]))

In [15]:
revenues = pd.concat(revenues,axis=1).T

In [16]:
revenues.index = pd.MultiIndex.from_arrays([climate_years, target_years])

In [17]:
costs = pd.read_csv("../technology-data/outputs/costs_2025.csv", index_col=[0,1])

In [18]:
eva_technologies = ['CCGT', 'OCGT', 'oil', 'biomass', 'lignite', 'coal']

In [19]:
costs = costs.reindex(eva_technologies, level=0)

In [20]:
average_expected_revenue = revenues.groupby(level=1).mean()

In [21]:
average_expected_revenue = average_expected_revenue[[i for i in average_expected_revenue.columns if i.split(" ")[1] in eva_technologies]]

In [22]:
start_year = revenues.index.levels[1][0]
end_year = revenues.index.levels[1][-1]

In [23]:
revenue_ratios_new = revenue_ratios_new_plants(average_expected_revenue)

In [24]:
demand_scaling_factor_new = (
        demand.groupby(level=2).mean()
        .reindex(revenue_ratios_new.index.str[:4])**0.5
    )

demand_scaling_factor_new.index = revenue_ratios_new.index

In [25]:
capacity_adjustment_new = scaling(
        revenue_ratios_new,
        offset,
        adjustment_factor    
    ).multiply(
        demand_scaling_factor_new
    )

In [27]:
revenue_ratios_existing = revenue_ratios_existing_plants(average_expected_revenue)

In [28]:
demand_scaling_factor_existing = (
        demand.groupby(level=2).mean()
        .reindex(revenue_ratios_existing.columns.str[:4])**0.5
    )

demand_scaling_factor_existing.index = revenue_ratios_existing.columns

In [29]:
capacity_adjustment_existing = scaling(
    revenue_ratios_existing,
    offset,
    adjustment_factor    
).multiply(
    demand_scaling_factor_existing
)

In [ ]:
p_nom_initial = initial_capacity_table.p_nom.unstack(0)

In [ ]:
p_nom_min = initial_capacity_table.p_nom_min.unstack(0)

In [ ]:
p_nom_max = initial_capacity_table.p_nom_max.unstack(0)

In [ ]:
next_capacities_new = pd.concat(
    [
        p_nom_initial[capacity_adjustment_new.index].add(capacity_adjustment_new).unstack(),
        p_nom_min[capacity_adjustment_new.index].unstack()
    ],
    axis=1
).max(axis=1).dropna()

In [ ]:
capacity_adjustment_existing = (
    capacity_adjustment_existing
    .reindex(capacity_adjustment_existing.index[::-1])
    .cummax()
    .sort_index()
)

In [ ]:
next_capacities_existing = pd.concat(
    [
        p_nom_initial[capacity_adjustment_existing.columns].add(capacity_adjustment_existing).unstack(),
        p_nom_max[capacity_adjustment_existing.columns].unstack()
    ],
    axis=1
).min(axis=1)

In [ ]:
capacity_adjustment_existing.filter(like="coal")

In [ ]:
next_capacities_existing = pd.concat(
    [
        next_capacities_existing,
        p_nom_min[capacity_adjustment_existing.columns].unstack(),
    ],
    axis=1
).max(axis=1)

In [ ]:
next_capacity_table = initial_capacity_table.copy()

In [ ]:
next_capacities_existing.dropna(inplace=True)

In [ ]:
next_capacities_new.dropna(inplace=True)

In [ ]:
next_capacity_table.loc[next_capacities_existing.index, "p_nom"] = next_capacities_existing

In [ ]:
next_capacity_table.loc[next_capacities_new.index, "p_nom"] = next_capacities_new

In [43]:
save_path_rr_exist = "results/revenue_ratios/1_existing.csv"
save_path_rr_new = "results/revenue_ratios/1_new.csv"

In [ ]:
os.makedirs(os.path.dirname(save_path_rr_exist), exist_ok=True)

In [42]:
revenue_ratios_existing.to_csv(save_path_rr_exist)
revenue_ratios_new.to_frame().to_csv(save_path_rr_new)

,AL00 CCGT exit 2034,AL00 OCGT exit 2034,AL00 oil exit 2028,AT00 CCGT exit 2028,AT00 CCGT exit 2030,AT00 CCGT exit 2033,AT00 CCGT exit 2034,AT00 biomass exit 2034,AT00 oil exit 2034,BA00 lignite exit 2028,...,UK00 OCGT exit 2034,UK00 biomass exit 2034,UK00 oil exit 2028,UK00 oil exit 2030,UK00 oil exit 2033,UK00 oil exit 2034,UKNI CCGT exit 2034,UKNI OCGT exit 2034,UKNI biomass exit 2034,UKNI oil exit 2034
2025,-28.524277,-31.884863,-566.215707,-257.572947,-0.809575,-363.772575,-605.623719,2801.608335,-515.456032,30.531050,...,-12915.386326,7141.934012,-4388.494741,-1568.760887,-1339.877572,-463.513039,-678.779683,-651.286432,411.189146,-2181.244868
2026,-28.520762,-31.945368,-568.452377,-272.926226,-0.859433,-382.544849,-638.252546,2763.162047,-526.679889,29.401430,...,-13066.651496,6884.276570,-4469.944102,-1597.876696,-1364.745363,-472.115725,-687.988016,-660.084623,401.797325,-2231.067628
2027,-28.738592,-32.123973,-572.832031,-277.326994,-0.872504,-388.166987,-648.690203,2744.732478,-531.087337,27.290034,...,-13115.615109,6788.218057,-4497.660905,-1607.784657,-1373.207746,-475.043175,-690.113307,-662.657344,398.283922,-2247.102175
